In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lifelines as lf
import sklearn as sk

In [2]:
# let's get our data
data = pd.read_csv('clean/afib_425_2_clean.csv')
data.describe()
T, E = data['duration'], data['event']

In [3]:
data.head()

,duration,event,age1,sex_0.0,sex_1.0,race_1.0,race_2.0,race_3.0,race_4.0,race_5.0,race_6.0,race_9.0
0,203.0,1.0,88.0,0,1,0,0,0,0,0,1,0
1,1076.0,1.0,67.0,1,0,1,0,0,0,0,0,0
2,1374.0,1.0,87.0,0,1,0,0,0,1,0,0,0
3,1074.0,1.0,87.0,1,0,1,0,0,0,0,0,0
4,1980.0,1.0,73.0,0,1,1,0,0,0,0,0,0


In [4]:
data['event'].value_counts()

0.0    526707
1.0     32541
Name: event, dtype: int64

In [5]:
from lifelines.utils.sklearn_adapter import sklearn_adapter
X = pd.DataFrame(data.drop('duration', axis=1))
Y = pd.DataFrame(data['duration'])

from sklearn import preprocessing
X = pd.DataFrame(preprocessing.scale(X))

In [6]:
from sklearn import model_selection
trainX, valX, trainY, valY = model_selection.train_test_split(X, Y)
trainX, valX, trainY, valY = pd.DataFrame(trainX), pd.DataFrame(valX), pd.DataFrame(trainY), pd.DataFrame(valY)
trainX.columns=['event','age','s1','s2','r1','r2','r3','r4','r5','r6','r9'] 

In [7]:
CoxRegression = sklearn_adapter(lf.CoxPHFitter, event_col='event')

sk_cph = CoxRegression(penalizer=.05)
sk_cph.fit(trainX, trainY)
sk_cph.lifelines_model.print_summary()

In [8]:
valX.columns=['event','age','s1','s2','r1','r2','r3','r4','r5','r6','r9'] 
sk_cph.score(valX, valY)

0.6104776720586487

In [9]:
from sklearn.linear_model import LinearRegression

lrg = LinearRegression()
lrg.fit(trainX, trainY)
print(lf.utils.concordance_index(valY, lrg.predict(valX), event_observed=valX['event']))

0.6138784749881531


In [10]:
WeibullAFT = sklearn_adapter(lf.WeibullAFTFitter, event_col='event')

sk_aft = WeibullAFT()
sk_aft.fit(trainX, trainY)
sk_aft.lifelines_model.print_summary()

'''
aft = lf.WeibullAFTFitter()
aft.fit(data, duration_col='duration', event_col='event')
aft.print_summary(3)
'''

C:\Users\tug84786\AppData\Local\Continuum\miniconda3\lib\site-packages\lifelines\fitters\__init__.py:1953: StatisticalWarning: The diagonal of the variance_matrix_ has negative values. This could be a problem with WeibullAFTFitter's fit to the data.

It's advisable to not trust the variances reported, and to be suspicious of the fitted parameters too.

  warnings.warn(warning_text, utils.StatisticalWarning)


"\naft = lf.WeibullAFTFitter()\naft.fit(data, duration_col='duration', event_col='event')\naft.print_summary(3)\n"

In [11]:
sk_aft.score(valX, valY)

0.6103734225733197

In [24]:
aff = lf.AalenAdditiveFitter(coef_penalizer=.05)
aff.fit(data, duration_col='duration', event_col='event')
aff.print_summary()